In [9]:
def loadSamples(samples):
    '''
    Load the sample data
    '''
    import csv
    with open('../data/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    return samples

In [10]:
samples = []
samples = loadSamples(samples)
print(len(samples))

8037


In [13]:
import cv2
def generator(samples, batch_size=32):
    '''
    Generate shuffled batch samples on the fly
    '''
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            steerings = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_steering = float(batch_sample[3])
                images.append(center_image)
                steerings.append(center_steering)

            # trim image to only see section with road
            X = np.array(images)
            y = np.array(steerings)
            yield sklearn.utils.shuffle(X, y)

In [20]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))

Train samples: 6429
Validation samples: 1608


In [21]:
# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

In [41]:
def nvidiaCNN(input_shape):
    '''
    Define the Nvidia End-to-End CNN architecture
    '''
    from keras.models import Sequential, Model
    from keras.layers import Lambda, Cropping2D, Convolution2D, Flatten, Dense
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape)) # First normalize the input image with Lambda layer
    model.add(Cropping2D(cropping=((70,25), (0,0)))) # Crop the usefule area
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu')) # 1st Convolution layer
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu')) # 2nd Convolution layer
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu')) # 3rd Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu')) # 4th Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu')) # 5th Convolution layer
    model.add(Flatten()) # Flatten layer
    model.add(Dense(100)) # 1st Fully connected layer
    model.add(Dense(50)) # 2nd Fully connected layer
    model.add(Dense(10)) # 3rd Fully connected layer
    model.add(Dense(1)) # Output layer
    return model


In [42]:
raw_image_shape = (160,320,3)
model = nvidiaCNN(raw_image_shape)